 #                            # HAND WRITTEN DIGIT RECOGNISATION

import libaries

In [8]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

add datasets

In [16]:
df_train = pd.read_csv(r"C:\Users\deepa\Downloads\hand written recognansation\train.csv\train.csv")
df_test = pd.read_csv(r"C:\Users\deepa\Downloads\hand written recognansation\test.csv\test.csv")

In [17]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Splitting into training and validation dataset

In [19]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [22]:
from sklearn.model_selection import train_test_split

# Assuming df_features and df_label are pandas DataFrames
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, test_size=0.2, random_state=1212)

# Use the values attribute instead of as_matrix
X_train = X_train.values.reshape(33600, 784)
X_cv = X_cv.values.reshape(8400, 784)

# Assuming X_test is a pandas DataFrame
X_test = X_test.values.reshape(28000, 784)


In [23]:
print((min(X_train[1]), max(X_train[1])))

(0, 255)


In [24]:
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [25]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [26]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [27]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [29]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_4 (Dense)      (None, 200)               20200     
                                                                 
 Output_Layer (Dense)        (None, 10)                2010      
                                                                 
Total params: 297910 (1.14 MB)
Trainable params: 297910 (1.14

In [30]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

In [31]:
# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [32]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 6s - loss: 1.8454 - accuracy: 0.5289 - val_loss: 1.0063 - val_accuracy: 0.7537 - 6s/epoch - 17ms/step
Epoch 2/20
336/336 - 3s - loss: 0.6600 - accuracy: 0.8229 - val_loss: 0.4829 - val_accuracy: 0.8633 - 3s/epoch - 10ms/step
Epoch 3/20
336/336 - 4s - loss: 0.4208 - accuracy: 0.8807 - val_loss: 0.3760 - val_accuracy: 0.8946 - 4s/epoch - 11ms/step
Epoch 4/20
336/336 - 3s - loss: 0.3403 - accuracy: 0.9024 - val_loss: 0.3219 - val_accuracy: 0.9071 - 3s/epoch - 10ms/step
Epoch 5/20
336/336 - 4s - loss: 0.2981 - accuracy: 0.9142 - val_loss: 0.3024 - val_accuracy: 0.9137 - 4s/epoch - 11ms/step
Epoch 6/20
336/336 - 3s - loss: 0.2699 - accuracy: 0.9215 - val_loss: 0.2654 - val_accuracy: 0.9225 - 3s/epoch - 10ms/step
Epoch 7/20
336/336 - 4s - loss: 0.2474 - accuracy: 0.9284 - val_loss: 0.2518 - val_accuracy: 0.9275 - 4s/epoch - 10ms/step
Epoch 8/20
336/336 - 4s - loss: 0.2293 - accuracy: 0.9323 - val_loss: 0.2347 - val_accuracy: 0.9310 - 4s/epoch - 11ms/step
Epoch 9/20
336/3

In [33]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [34]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 7s - loss: 0.3412 - accuracy: 0.8978 - val_loss: 0.1574 - val_accuracy: 0.9535 - 7s/epoch - 20ms/step
Epoch 2/20
336/336 - 4s - loss: 0.1216 - accuracy: 0.9631 - val_loss: 0.1245 - val_accuracy: 0.9602 - 4s/epoch - 12ms/step
Epoch 3/20
336/336 - 4s - loss: 0.0790 - accuracy: 0.9750 - val_loss: 0.1040 - val_accuracy: 0.9677 - 4s/epoch - 12ms/step
Epoch 4/20
336/336 - 4s - loss: 0.0589 - accuracy: 0.9813 - val_loss: 0.1272 - val_accuracy: 0.9636 - 4s/epoch - 13ms/step
Epoch 5/20
336/336 - 4s - loss: 0.0469 - accuracy: 0.9843 - val_loss: 0.1144 - val_accuracy: 0.9686 - 4s/epoch - 13ms/step
Epoch 6/20
336/336 - 4s - loss: 0.0326 - accuracy: 0.9895 - val_loss: 0.1104 - val_accuracy: 0.9699 - 4s/epoch - 12ms/step
Epoch 7/20
336/336 - 4s - loss: 0.0285 - accuracy: 0.9911 - val_loss: 0.0913 - val_accuracy: 0.9742 - 4s/epoch - 12ms/step
Epoch 8/20
336/336 - 4s - loss: 0.0253 - accuracy: 0.9919 - val_loss: 0.1126 - val_accuracy: 0.9726 - 4s/epoch - 11ms/step
Epoch 9/20
336/3

In [35]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 7s - loss: 0.3491 - accuracy: 0.8965 - val_loss: 0.2055 - val_accuracy: 0.9388 - 7s/epoch - 21ms/step
Epoch 2/20
336/336 - 4s - loss: 0.1208 - accuracy: 0.9622 - val_loss: 0.1216 - val_accuracy: 0.9618 - 4s/epoch - 12ms/step
Epoch 3/20
336/336 - 4s - loss: 0.0792 - accuracy: 0.9749 - val_loss: 0.1127 - val_accuracy: 0.9669 - 4s/epoch - 12ms/step
Epoch 4/20
336/336 - 4s - loss: 0.0575 - accuracy: 0.9815 - val_loss: 0.1254 - val_accuracy: 0.9636 - 4s/epoch - 12ms/step
Epoch 5/20
336/336 - 4s - loss: 0.0444 - accuracy: 0.9854 - val_loss: 0.0925 - val_accuracy: 0.9760 - 4s/epoch - 12ms/step
Epoch 6/20
336/336 - 4s - loss: 0.0352 - accuracy: 0.9888 - val_loss: 0.1187 - val_accuracy: 0.9696 - 4s/epoch - 12ms/step
Epoch 7/20
336/336 - 4s - loss: 0.0306 - accuracy: 0.9902 - val_loss: 0.1098 - val_accuracy: 0.9712 - 4s/epoch - 12ms/step
Epoch 8/20
336/336 - 4s - loss: 0.0270 - accuracy: 0.9911 - val_loss: 0.1071 - val_accuracy: 0.9744 - 4s/epoch - 12ms/step
Epoch 9/20
336/3

In [37]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [38]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 8s 15ms/step - loss: 0.3367 - accuracy: 0.8957 - val_loss: 0.1662 - val_accuracy: 0.9512
Epoch 2/20
336/336 [==============================] - 4s 13ms/step - loss: 0.1244 - accuracy: 0.9615 - val_loss: 0.1301 - val_accuracy: 0.9608
Epoch 3/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0795 - accuracy: 0.9748 - val_loss: 0.1324 - val_accuracy: 0.9613
Epoch 4/20
336/336 [==============================] - 4s 12ms/step - loss: 0.0608 - accuracy: 0.9805 - val_loss: 0.0971 - val_accuracy: 0.9700
Epoch 5/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0447 - accuracy: 0.9862 - val_loss: 0.1084 - val_accuracy: 0.9693
Epoch 6/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0375 - accuracy: 0.9879 - val_loss: 0.1111 - val_accuracy: 0.9713
Epoch 7/20
336/336 [==============================] - 4s 12ms/step - loss: 0.0309 - accuracy: 0.9898 - val_loss: 0.0968 - val_accuracy: 0.9756

In [39]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [40]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [41]:
# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_4 (Dense)      (None, 100)               10100     
                                                                 
 Hidden_Layer_5 (Dense)      (None, 200)               20200     
                                                                 
 Output_Layer (Dense)        (None, 10)                2010

In [42]:
# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [43]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 7s 14ms/step - loss: 0.3519 - accuracy: 0.8911 - val_loss: 0.1598 - val_accuracy: 0.9530
Epoch 2/20
336/336 [==============================] - 4s 12ms/step - loss: 0.1232 - accuracy: 0.9611 - val_loss: 0.1296 - val_accuracy: 0.9598
Epoch 3/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0853 - accuracy: 0.9742 - val_loss: 0.1159 - val_accuracy: 0.9651
Epoch 4/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0635 - accuracy: 0.9805 - val_loss: 0.0902 - val_accuracy: 0.9718
Epoch 5/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0477 - accuracy: 0.9851 - val_loss: 0.1077 - val_accuracy: 0.9689
Epoch 6/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0384 - accuracy: 0.9876 - val_loss: 0.0968 - val_accuracy: 0.9735
Epoch 7/20
336/336 [==============================] - 4s 13ms/step - loss: 0.0342 - accuracy: 0.9891 - val_loss: 0.1023 - val_accuracy: 0.9733

In [44]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [45]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [46]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 784)]             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 300)               235500    
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 Hidden_Layer_2 (Dense)      (None, 100)               30100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 Hidden_Layer_3 (Dense)      (None, 100)               10100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0   

In [47]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [48]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 [==============================] - 10s 15ms/step - loss: 0.5854 - accuracy: 0.8140 - val_loss: 0.1997 - val_accuracy: 0.9404
Epoch 2/20
336/336 [==============================] - 5s 13ms/step - loss: 0.2333 - accuracy: 0.9317 - val_loss: 0.1386 - val_accuracy: 0.9600
Epoch 3/20
336/336 [==============================] - 4s 13ms/step - loss: 0.1745 - accuracy: 0.9498 - val_loss: 0.1129 - val_accuracy: 0.9674
Epoch 4/20
336/336 [==============================] - 5s 13ms/step - loss: 0.1387 - accuracy: 0.9590 - val_loss: 0.1108 - val_accuracy: 0.9696
Epoch 5/20
336/336 [==============================] - 5s 14ms/step - loss: 0.1221 - accuracy: 0.9650 - val_loss: 0.1003 - val_accuracy: 0.9714
Epoch 6/20
336/336 [==============================] - 5s 14ms/step - loss: 0.1112 - accuracy: 0.9674 - val_loss: 0.0996 - val_accuracy: 0.9692
Epoch 7/20
336/336 [==============================] - 5s 14ms/step - loss: 0.0953 - accuracy: 0.9727 - val_loss: 0.0959 - val_accuracy: 0.974

In [54]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred

140/140 [==============================] - 1s 7ms/step


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [50]:
test_pred.to_csv('mnist_submission.csv', index = False)